In [3]:
import os
from os.path import isfile,join

import numpy as np
import matplotlib.pyplot as plt
import re

import cv2

In [8]:
frames = os.listdir('frames/')
frames.sort(key=lambda frame: int(re.sub('\D','',frame)))

images = []

for frame in frames:
    img = cv2.imread('frames/'+frame)
    images.append(img)